# Projekt z przedmiotu Podstawy Reprezentacji i Analizy Danych

## Temat Projektu - Twitter

Projekt dotyczy zbioru danych z Twittera, zawierającego profile i przykładowe tweety użytkowników. Celem naszego projektu jest analiza podanego zbioru danych i próba przewidzenia płci (lub przynależności do firmy) użytkownika na jego podstawie.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# zmiana sposobu wyświetlania danych typu float
pd.options.display.float_format = "{:.2f}".format

import re
import time
import collections

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize 

## Opis zbioru danych

Zbiór danych składa się z kilkunastu kolumn z danymi. Są to:
1. __\_unit_id__: unikalne id użytkownika
2. __\_golden__: czy użytkownik znajduje się w złotym standardzie dla modelu; TRUE lub FALSE
3. __\_unit_state__: stan obserwacji
4. __\_trusted_judgments__: liczba potwierdzonych osądów
5. __\_last_judgment_at__: data i godzina ostatniego potwierdzonego osądu
6. __gender__: płeć lub przynależność do firmy, odpowiednio(male, female, lub brand (dla firm))
7. __gender\:confidence__: liczba rzeczywista z zakresu [0 , 1] odpowiadająca pewności osądu przynależności do płci lub firmy
8. __profile_yn__: informacja czy dany profil był dostępny do osądu
9. __profile_yn\:confidence__: pewność czy dany profil istnieje czy nie
10. __created__: data i czas utworzenia profilu
11. __description__: opis profilu użytkownika
12. __fav_number__: liczba twwetów, które użytkownik polubił
13. __gender_gold__: jaka jest płeć, jeżeli profil znajduje się w złotym standardzie
14. __link_color__: kolor linku w formacie hex
15. __name__: nazwa użytkownika
16. __profile_yn_gold__: czy pole __profile_yn__ należy do złotego standardu
17. __profileimage__: link do zdjęcia profilowego
18. __retweet_count__: ile razu użytkownik podał dalej tweeta
19. __sidebar_color__: kolor paska bocznego jako wartość hex
20. __text__: tekst jednego losowego tweetu użytkownika
21. __tweet_coord__: pozycja geograficzna użytkownika, jeśli była dostępna, w postaci "[szerokość geograficzna, długość geograficzna]"
22. __tweet_count__: liczba twwetów użytkownika
23. __tweet_created__: kiedy tweet z kolumny __text__ został utworzony
24. __tweet_id__: id losowego tweeta
25. __tweet_location__: lokalizacja tweeta
26. __user_timezone__: strefa czasowa użytkownika

## Wczytanie danych

Dane przechowywane są w pliku csv o nazwie __gender-classifier-DFE-791531.csv__. Za pomocą pakietu pandas wczytaliśmy je do naszego skryptu. Zdecydowaliśmy się na wybór kolumn 6,7, 11, 12, 14, 15, 18, 19, 20, 22, ponieważ inne kolumny zawierają informacje nieprzydatne dla naszego problemu.

In [ ]:
df = pd.read_csv("../Data/gender-classifier-DFE-791531.csv", 
                 encoding = "latin1", 
                 engine='python', 
                 usecols=[5, 6, 10, 11, 13, 14, 17, 18, 19, 21])
df.head()

We wczytanym zbiorze danych znajduje się 20050 obiektów. Posiadają one 10 atrybutów, z których 6 jest typu `object`, 3 są typu `int64` i 1 typu `float64`. 

In [ ]:
df.info()

W zbiorze danych występują braki. Najwięcej wartości nieznanych jest w kolumnie `description`, która zawiera opis profilu użytkownika.

In [ ]:
df.isna().sum(axis=0)

## Czyszczenie danych

Kolumna `gender:confidence` zawiera pewność osądzenia płci użytkownika, przyjmuje ona wartości od 0 do 1. Do dalszej analizy usuniemy obiekty z pewnością mniejszą niż __0.9__ lub nieznaną, ze względu na ich małą wiarygodność.

In [ ]:
df = df.where((df['gender:confidence'] > 0.9) & (df['gender'] != 'unknown')).dropna()
df.head()

Po usunięciu wyżej wspomnianych obiektów w zbiorze danych pozostało 11779 wierszy. Nie ma w nich już braków.

In [ ]:
df.info()

Po wykonaniu redukcji można usunąć kolumnę `gender:confidence`, ponieważ nie będzie nam ona potrzebna w dalszej analizie.

In [ ]:
df = df.drop(columns=['gender:confidence'])

## Przekształcenie kolumn zawierających kolory

W zbiorze danych są dwie kolumny zawierające informacje o kolorach. Są to: `link_color` oraz `sidebar_color`. Kolory są w nich zapisane heksadecymalnie. W celu użycia kolorów podczas klasyfikacji trzeba przekonwertować je do postaci RGB.

W kolumnach opisujących kolor występują wartości `9999` oraz `0`. Nie są one w formacie heksadecymalnym, więc należy je na niego zamienić.

In [ ]:
df.link_color = df.link_color.replace(['9999', '0'], '000000')
df.sidebar_color = df.sidebar_color.replace(['9999', '0'], '000000')

Funkcja `hexToRGB` dokonuje konwersji liczby zapisanej w formacie heksadecymalnym na postać RGB.

In [ ]:
def hexToRGB(color):
    if color == '0':
        return 255, 255, 255
    if len(color)<5:
        return 255, 255, 255
    try:
        color = color.replace(".", "")
        color = color.replace("+", "")
        r=int(color[0:2],16)
        g=int(color[2:4],16)
        b=int(color[4:6],16)
    except (RuntimeError, TypeError, NameError, ValueError):
        return None, None, None
    else:
        return r,g,b 

Dodanie kolumn z wartościami RGB kolorów w kolumnach `link_color` i `sidebar_color`.

In [ ]:
df["link_color_R"] = df["link_color"].apply(lambda x: hexToRGB(str(x))[0])
df["link_color_G"] = df["link_color"].apply(lambda x: hexToRGB(str(x))[1])
df["link_color_B"] = df["link_color"].apply(lambda x: hexToRGB(str(x))[2])

df["sidebar_color_R"] = df["sidebar_color"].apply(lambda x: hexToRGB(str(x))[0])
df["sidebar_color_G"] = df["sidebar_color"].apply(lambda x: hexToRGB(str(x))[1])
df["sidebar_color_B"] = df["sidebar_color"].apply(lambda x: hexToRGB(str(x))[2])

df = df.dropna()
df.loc[:, ['link_color', 'sidebar_color', 'link_color_R', 'link_color_G', 'link_color_B', 'sidebar_color_R', 'sidebar_color_G', 'sidebar_color_B']].head()

## Wykresy przedstawiające najczęściej używane kolory

In [ ]:
male_top_link_color = df[df['gender'] == 'male']['link_color'].value_counts().head(10)
male_top_link_color_idx = male_top_link_color.index
male_top_color = male_top_link_color_idx.values

l = lambda x: '#'+x

sns.set_style("darkgrid", {"axes.facecolor": "#E0D6D6"})
sns.barplot(x = male_top_link_color, y = male_top_color , palette=list(map(l, male_top_color))).set_title('Most used link colors by men')

In [ ]:
female_top_link_color = df[df['gender'] == 'female']['link_color'].value_counts().head(10)
female_top_link_color_idx = female_top_link_color.index
female_top_color = female_top_link_color_idx.values

l = lambda x: '#'+x

sns.set_style("darkgrid", {"axes.facecolor": "#E0D6D6"})
sns.barplot (x = female_top_link_color, y = female_top_color , palette=list(map(l, female_top_color))).set_title('Most used link colors by women')

In [ ]:
brand_top_link_color = df[df['gender'] == 'brand']['link_color'].value_counts().head(10)
brand_top_link_color_idx = brand_top_link_color.index
brand_top_color = brand_top_link_color_idx.values

l = lambda x: '#'+x

sns.set_style("darkgrid", {"axes.facecolor": "#E0D6D6"})
sns.barplot (x = brand_top_link_color, y = brand_top_color , palette=list(map(l, brand_top_color))).set_title('Most used link colors by brands')

Najczęściej wybieranym kolorem linku przez użytkowników Twittera jest kolor niebieski. W przypadku kolejnych kolorów kolejności występują różnicę. Drugim najczęściej wybieranym kolorem jest fioletowy dla kobiet, inny odcień niebieskiego dla mężczyzn oraz zielony dla firm. Ze względu na różnicę w wykresach, kolor linku jako atrybut może być przydatny.

In [ ]:
male_top_sidebar_color = df[df['gender'] == 'male']['sidebar_color'].value_counts().head(10)
male_top_sidebar_color_idx = male_top_sidebar_color.index
male_top_color = male_top_sidebar_color_idx.values

l = lambda x: '#'+x

sns.set_style("darkgrid", {"axes.facecolor": "#E0D6D6"})
sns.barplot (x = male_top_sidebar_color, y = male_top_color , palette=list(map(l, male_top_color))).set_title('Most used sidebar colors by men')

In [ ]:
female_top_sidebar_color = df[df['gender'] == 'female']['sidebar_color'].value_counts().head(10)
female_top_sidebar_color_idx = female_top_sidebar_color.index
female_top_color = female_top_sidebar_color_idx.values

l = lambda x: '#'+x

sns.set_style("darkgrid", {"axes.facecolor": "#E0D6D6"})
plot4 =sns.barplot (x = female_top_sidebar_color, y = female_top_color, palette=list(map(l, female_top_color))).set_title('Most used sidebar colors by women')

In [ ]:
brand_top_sidebar_color = df[df['gender'] == 'brand']['sidebar_color'].value_counts().head(10)
brand_top_sidebar_color_idx = brand_top_sidebar_color.index
brand_top_color = brand_top_sidebar_color_idx.values

l = lambda x: '#'+x

sns.set_style("darkgrid", {"axes.facecolor": "#E0D6D6"})
sns.barplot (x = brand_top_sidebar_color, y = brand_top_color , palette=list(map(l, brand_top_color))).set_title('Most used sidebar colors by brands')

Cztery najczęściej używane kolory paska bocznego są takie same dla mężczyzn, kobiet i firm. Są to niebieski, biały, czarny i szary. Ze względu na brak różnic między grupami, kolor paska bocznego nie będzie atrybutem przydatnym podczas decydowania o przynależności do grup.

## Wykresy przedstawiające dane z atrybutów ilościowych

W zbiorze danych znajdują się 3 atrybuty ilościowe. Są to `fav_number`, `tweet_count`, `retweet_count`.

### Ilość polubień użytkownika

In [ ]:
gfn_mean = (df.loc[:, ['gender', 'fav_number']]
            .groupby('gender')
            .mean()
            .sort_values(by='fav_number'))
gfn_mean

In [ ]:
sns.set_style("whitegrid", {"axes.facecolor": "#FFFFFF"})
gfn_mean.plot.bar()

### Ilość tweetów użytkownika

In [ ]:
gtc_mean = (df.loc[:, ['gender', 'tweet_count']]
            .groupby('gender')
            .mean()
            .sort_values(by='tweet_count'))
gtc_mean

In [ ]:
gtc_mean.plot.bar()

### Ilość razy kiedy użytkownik podał tweet dalej

In [ ]:
grc_mean = (df.loc[:, ['gender', 'retweet_count']]
            .groupby('gender')
            .mean()
            .sort_values(by='retweet_count'))
grc_mean

In [ ]:
grc_mean.plot.bar()

## Przekształcanie kolumn zawierających tekst

### Usuwanie znaków specjalnych z tekstu i opisu

Funckja usuwająca znaki spoza systemu ASCII, linki, znaki specjalne oraz podwójne spacje.

In [ ]:
def normalize_text(text):
    text = text.lower()
    
    # Usuń znaki spoza ASCII
    text = re.sub('[^\x00-\x7F]+',' ', text)
    
    # Usuń linki
    text = re.sub('https?:\/\/.*[\r\n] ', ' ', text)
    
    # Usuń znaki specjalne
    text = re.sub('[?!+%{}:;.,#"\'()\[\]_#\&-@]', '',text) #@&-
    
    # Usuń podwójne spacje
    text = re.sub('\s+',' ',text)
    
    return text

W orginalnej tabeli zostają stworzone trzy dodatkowe kolumny. Jedna `tekst_norm` jest znormalizowanym tekstem tweeta, druga `description_norm` jest znormalizowanym opisem oraz trzecia `all_text` jest połączeniem znormalizowanych tekstu i opisu.

In [ ]:
df['text_norm'] = [normalize_text(text) for text in df['text']]
df['description_norm'] = [normalize_text(text) for text in df['description']]
df['all_text'] = df['text_norm'].str.cat(df['description_norm'],sep=' ')

df = df.reset_index(drop=True)

### Stop words

Zbiór słów nazywany __stop words__ zawiera słowa nieposiadające dużego znaczenia. Usuwa się je w celu oczyszczenia danych tekstowych.

In [ ]:
def del_stop_words_from_text(text):
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(text) 
    
    w_text = ""
  
    for w in word_tokens: 
        if w not in stop_words: 
            w_text += w
            w_text += " "
    
    return w_text

In [ ]:
df['text_stop'] = [del_stop_words_from_text(text) for text in df['text_norm']]
df['all_text_stop'] = [del_stop_words_from_text(text) for text in df['all_text']]
df['description_stop'] = [del_stop_words_from_text(text) for text in df['description_norm']]

Przykładowe tweety przed i po usunięciu __stop words__.

In [ ]:
df['text_norm'][3]

In [ ]:
df['text_stop'][3]

### Stemming

__Stemming__ to technika normalizacji danych tekstowych polegająca usunięciu końcówek z wyrazów.

In [ ]:
def stem_text(text, stemmer):
    text_list = text.split()
    
    s_text = ""
    
    for item in text_list:
        s_text += stemmer.stem(item)
        s_text += " "
    
    return s_text

In [ ]:
stemmer = SnowballStemmer("english")

df['text_stem'] = [stem_text(text, stemmer) for text in df['text_stop']]
df['all_text_stem'] = [stem_text(text, stemmer) for text in df['all_text_stop']]
df['description_stem'] = [stem_text(text, stemmer) for text in df['description_stop']]

Przykładowe tweety przed i po zastosowaniu techniki __stemming__.

In [ ]:
df['text_stop'][3]

In [ ]:
df['text_stem'][3]

## Wizualizacja najczęściej używanych słów ze względu na płeć

Funckja tworząca wykresy najczęściej użytych słów dla płci oraz dla wybranego tekstu. 

In [ ]:
def plot_common_words(gender, text_name, color):
    X = df[df['gender'] == gender][[text_name]]
    X.columns=['Text']
    text = X['Text'].tolist()

    tokens = []

    for item in text:
        tokens.append(word_tokenize(item))

    words = []

    for item in tokens:
        words = words + item

    counter = dict(collections.Counter(words))
    counter_df = pd.DataFrame.from_dict(counter, orient='index').reset_index().rename(columns={'index':'word', 0:'count'})
    counter_df = counter_df.sort_values(by=['count'], ascending=False).head(10)
    
    plt.figure(figsize=(14, 7))
    plt.barh(counter_df['word'], counter_df['count'], color=color)
    plt.title('Most used words by ' + gender + 's for ' + text_name)
    plt.xlabel('frequency')
    plt.ylabel('words')
    plt.gca().invert_yaxis()
    plt.show()

Użycie funkcji, dla wszystkich płci, dla połączonego tekstu tweeta i opisu.

In [ ]:
# cały tekst
plot_common_words('male', 'all_text_stem', '#0C86CF')
plot_common_words('female', 'all_text_stem', '#E6ADD8')
plot_common_words('brand', 'all_text_stem', '#000000')

W połączonym tekście widać wyraźnie różnicę w słowach używanych przez firmy i przez osoby prywatne. Niestety różnica między słowami używanymi przez mężczyzn i kobiety.

In [ ]:
# opis
plot_common_words('male', 'description_stem', '#0C86CF')
plot_common_words('female', 'description_stem', '#E6ADD8')
plot_common_words('brand', 'description_stem', '#000000')

W opisie profilu ponownie można wyraźnie odróżnić słowa używane przez firmy. W przypadku obu płci pokrywają się pierwsze 4 pozycje na liście, jednak kolejne słowa różnią się od siebie.

In [ ]:
# tekst tweeta
plot_common_words('male', 'text_stem', '#0C86CF')
plot_common_words('female', 'text_stem', '#E6ADD8')
plot_common_words('brand', 'text_stem', '#000000')

W tekscie tweeta sytuacja jest podobna jak w przypadku całego tekstu i opisu.

# Przewidywanie płci

## Wybór klasyfikatora

Postanowiliśmy sprawdzić działanie trzech klasyfikatorów: `MultinomialNB`, `KNeighborsClassifier` i `RandomForestClassifier`, czyli odpowiednio naiwnego klasyfikatora bayesowskiego, klasyfikatora k-najbliższych sąsiadów oraz klasyfikatora wykorzystującego drzewa decyzyjne.

Funkcja wykorzystująca podany klasyfikator, przewidująca jakiej płci jest użytkownik, na podstawie wybranego tekstu.

In [ ]:
def predict_for_set(data_set, text, clf):
    X = data_set[[text]]
    y = data_set[['gender']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    count_vect = CountVectorizer()
    X_train_vect = count_vect.fit_transform(X_train.iloc[:, 0])
    X_test_vect = count_vect.transform(X_test.iloc[:, 0])

    
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_vect)
    X_test_tfidf = tfidf_transformer.fit_transform(X_test_vect)

    t_start=time.time()
    
    alg = clf.fit(X_train_tfidf, y_train)

    t_pred = round(time.time() - t_start, 3)
    
    y_pred = alg.predict(X_test_tfidf)
    
    y_pred_tr = alg.predict(X_train_tfidf)

    accuracy = metrics.accuracy_score(y_test, y_pred)
    
    return accuracy, t_pred

Porównanie klasyfikatorów i różnych tekstów.

In [ ]:
clf_m = MultinomialNB()
clf_k = KNeighborsClassifier()
clf_r = RandomForestClassifier()

clf_list = [clf_m, 
            clf_k, 
            clf_r]

clf_names_list = ['MultinomialNB', 
                  'KNeighborsClassifier', 
                  'RandomForestClassifier']

texts_list = ['text_norm', 
              'description_norm', 
              'all_text']

data = {'Klasyfikator':[], 
        'Kolumna z tekstem':[], 
        'Dokładność':[], 
        'Czas':[]}

for i in range(len(clf_list)):
    for text_name in texts_list:
        res = predict_for_set(df, text_name, clf_list[i])
        data['Klasyfikator'].append(clf_names_list[i])
        data['Kolumna z tekstem'].append(text_name)
        data['Dokładność'].append(res[0])
        data['Czas'].append(res[1])
        
clf_df = pd.DataFrame(data=data)
clf_df

Wykres przedstawiający dokładność i czas trenowania w zależności od wybranego klasyfikatora i tekstu.

In [ ]:
clf_df.sort_values(by='Dokładność').plot.barh()

Na podstawnie wykresu widać, że najlepsze efekty daje klasyfikator `MultinomialNB` z wykorzystaniem kolumny `description_norm`, czyli samego opisu.	

In [ ]:
def predict_for_set_and_draw(data_set, text, clf):
    X = data_set[[text]]
    y = data_set[['gender']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    count_vect = CountVectorizer()
    X_train_vect = count_vect.fit_transform(X_train.iloc[:, 0])
    X_test_vect = count_vect.transform(X_test.iloc[:, 0])

    
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_vect)
    X_test_tfidf = tfidf_transformer.fit_transform(X_test_vect)
    
    alg = clf.fit(X_train_tfidf, y_train)
    
    y_pred = alg.predict(X_test_tfidf)
    
    y_pred_tr = alg.predict(X_train_tfidf)
    
    print("Accuracy dla zbioru testowego: ")
    print(metrics.accuracy_score(y_test, y_pred), "\n")
    print("Accuracy dla zbioru uczącego: ")
    print(metrics.accuracy_score(y_train, y_pred_tr), "\n")
    
    #print("Macierz pomyłek dla zbioru uczącego")
    #print(pd.crosstab(y_train.to_numpy().transpose()[0], y_pred_tr)) 
    #sns.heatmap(pd.crosstab(y_train.to_numpy().transpose()[0], y_pred_tr),
    #        cmap="YlGnBu", annot=True, cbar=False)
    
    print("Macierz pomyłek dla zbioru testowego")
    print(pd.crosstab(y_test.to_numpy().transpose()[0], y_pred)) 
    sns.heatmap(pd.crosstab(y_test.to_numpy().transpose()[0], y_pred),
            cmap="YlGnBu", annot=True, cbar=False)

Przewidywanie płci na podstawie opisu

In [ ]:
predict_for_set_and_draw(df, 'description_norm', clf_m)

Na podstawnie macierzy pomyłek widać, że klasyfikator poprawnie rozpoznaje firmy, problem ma z rozróżnieniem kobiet i mężczyzn.

## Podsumowanie

Problemem postawionym podczas tego projektu było rozpoznawanie płci użytkowników Twittera. Udało nam się rozpoznawać płeć z dokładnością około __67%__. Wynik ten mógłbybyć poprawiony prawdopodocnie poprzez dodanie większej ilości atrybutów lub zastosowanie bardziej złożonych obliczeniowo algorytmów.